In [79]:
with open("..\data\szoveg.txt", "r", encoding="utf8") as f:     #megnyitjuk a beolvasnadó fájlt, majd kisbetűssé alakítjuk azt, végül kiírjuk a hosszát
    text = f.read().lower()

print(len(text))

30584


In [80]:
text[:20000]

'hello, everyone! this is the longest text ever! i was inspired by the various other "longest texts ever" on the internet, and i wanted to make my own. so here it is! this is going to be a world record! this is actually my third attempt at doing this. the first time, i didn\'t save it. the second time, the neocities editor crashed. now i\'m writing this in notepad, then copying it into the neocities editor instead of typing it directly in the neocities editor to avoid crashing. it sucks that my past two attempts are gone now. those actually got pretty long. not the longest, but still pretty long. i hope this one won\'t get lost somehow. anyways, let\'s talk about waffles! i like waffles. waffles are cool. waffles is a funny word. there\'s a teen titans go episode called "waffles" where the word "waffles" is said a hundred-something times. it\'s pretty annoying. there\'s also a teen titans go episode about pig latin. don\'t know what pig latin is? it\'s a language where you take all the

In [81]:
import numpy as np
from tqdm import tqdm

maxlen = 100      #a kiválasztott stringek hossza
step = 2           #létrehozunk egy új sentencet 2 lépésenként
sentences = []     #tartalmazza a kiszedett sentenceket 
next_chars = []

for i in tqdm(range(0, len(text) - maxlen, step)):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
len(sentences)

100%|██████████| 15242/15242 [00:00<00:00, 1310408.35it/s]


15242

In [82]:
for i in range(5): print(repr(sentences[i])," : ", next_chars[i])

'hello, everyone! this is the longest text ever! i was inspired by the various other "longest texts e'  :  v
'llo, everyone! this is the longest text ever! i was inspired by the various other "longest texts eve'  :  r
'o, everyone! this is the longest text ever! i was inspired by the various other "longest texts ever"'  :   
' everyone! this is the longest text ever! i was inspired by the various other "longest texts ever" o'  :  n
'veryone! this is the longest text ever! i was inspired by the various other "longest texts ever" on '  :  t


In [83]:
chars = sorted(list(set(text)))  #a szöveg különleges karaktereit tartalmazza
print(chars)

[' ', '!', '"', '%', "'", '(', ')', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '~', '‘', '’', '“', '”']


In [84]:
char_indices = dict((char, chars.index(char)) for char in chars)  #mapba teszi el a karaktereket és azok indexeit
print(char_indices)

{' ': 0, '!': 1, '"': 2, '%': 3, "'": 4, '(': 5, ')': 6, '+': 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'a': 24, 'b': 25, 'c': 26, 'd': 27, 'e': 28, 'f': 29, 'g': 30, 'h': 31, 'i': 32, 'j': 33, 'k': 34, 'l': 35, 'm': 36, 'n': 37, 'o': 38, 'p': 39, 'q': 40, 'r': 41, 's': 42, 't': 43, 'u': 44, 'v': 45, 'w': 46, 'x': 47, 'y': 48, 'z': 49, '~': 50, '‘': 51, '’': 52, '“': 53, '”': 54}


In [85]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)        #a karaktereket bináris listává kódolja át
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in tqdm(enumerate(sentences)):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

print("x", x.shape)
print("y", y.shape)

C:\Users\jenei\AppData\Local\Temp\ipykernel_17740\293909962.py:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)        #a karaktereket bináris listává kódolja át
C:\Users\jenei\AppData\Local\Temp\ipykernel_17740\293909962.py:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zer

x (15242, 100, 55)
y (15242, 55)


In [86]:
import tensorflow as tf                  #build network

model = tf.keras.Sequential([
    tf.keras.layers.GRU(16, input_shape=(maxlen, len(chars))),
    tf.keras.layers.Dense(units=len(chars), activation='softmax')
]) 

model.compile(loss='categorical_crossentropy', 
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.01))

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_6 (GRU)                 (None, 16)                3504      
                                                                 
 dense_6 (Dense)             (None, 55)                935       
                                                                 
Total params: 4,439
Trainable params: 4,439
Non-trainable params: 0
_________________________________________________________________


In [87]:
with tf.device("/CPU:0"):
    model.fit(x, 
            y,
            batch_size=128,
            epochs=1)

120/120 [==============================] - 5s 26ms/step - loss: 2.9238


In [90]:
import random             #kiválaszt egy random sort amivel kezdődni fog a generált szöveg

start_index = random.randint(0, len(text) - maxlen - 1)
base_text = text[start_index: start_index + maxlen]
base_text

', and appearantly it\'s "i", with 59 uses! the word "i" makes up 5% of the words this text! i would\'v'

In [91]:
def sample(preds, temperature=1.0):     #betanul
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [92]:
import sys                             #a lényeg, a szöveg legenerálása
temperatures = [0.2, 0.5, 1.0, 1.2]
gen_characters = 400

for temp in temperatures:       #generálunk 400 karaktert különböző sebességgel
    print("Temp: ", temp)
    generated_text = base_text
    print(generated_text)
    for i in range(gen_characters):
        sampled = np.zeros((1, maxlen, len(chars)))    
        for t, char in enumerate(generated_text):
            sampled[0, t, char_indices[char]] = 1.          
        
        preds = model.predict(sampled, verbose=0)[0]      
        
        next_index = sample(preds, temp)
        next_char = chars[next_index]

        generated_text += next_char
        generated_text = generated_text[1:]

        sys.stdout.write(next_char)
        sys.stdout.flush()

    print()

Temp:  0.2
, and appearantly it's "i", with 59 uses! the word "i" makes up 5% of the words this text! i would'v
e the the the the an is the the an at the the the the the the the the an at the the the the thar at  at the the the the the than the the the the the the the the the the the it the the the the the the th at the the it the an the te the the the the the it the the the bin the than the the that e the the the the the the in an the the the the at that the the the an i the te the the the te the ar i the 
Temp:  0.5
, and appearantly it's "i", with 59 uses! the word "i" makes up 5% of the words this text! i would'v
en thns a topt ins ad the ons mhe the t te the inthot tha at i an al arat ol f a ind than ot that bong itharan age t the than it t at o tug tre te te uid at thant at y arre t thant athe iol tbin than e thentin t aa ia i the the at bon oe mit ba man ave oo ateng it ton y tout a e t re lh om gont ira as y am t ad went the it nbe ton th ann ot the int il ath ot at thim the th

KeyboardInterrupt: 